# FjordHQ IoS-001 Yahoo Finance Backfill
## Colab Script for Price Data Ingestion

**Purpose**: Backfill canonical assets with historical price data from Yahoo Finance

**Authority**: CEO Directive CD-IOS-001-PRICE-BACKFILL-001

**Rate Limiting Strategy**:
- 2 second delay between requests
- Batch processing with 60s cooldown between batches
- Automatic retry with exponential backoff

---

In [ ]:
# Cell 1: Install dependencies
!pip install yfinance psycopg2-binary pandas --quiet
print("Dependencies installed.")

In [ ]:
# Cell 2: Configuration
# IMPORTANT: Update these values before running!

DB_CONFIG = {
    'host': 'YOUR_SUPABASE_HOST',  # e.g., 'db.xxx.supabase.co'
    'port': 5432,
    'database': 'postgres',
    'user': 'postgres',
    'password': 'YOUR_PASSWORD'
}

# Which asset classes to backfill?
BACKFILL_CRYPTO = True
BACKFILL_EQUITY = False  # Set True to include equities
BACKFILL_FX = False      # Set True to include FX

# Rate limiting settings
DELAY_BETWEEN_REQUESTS = 2  # seconds
BATCH_SIZE = 10
DELAY_BETWEEN_BATCHES = 60  # seconds

# How much history?
START_DATE = '2010-01-01'  # Max history for most assets

print(f"Configuration loaded.")
print(f"Crypto: {BACKFILL_CRYPTO}, Equity: {BACKFILL_EQUITY}, FX: {BACKFILL_FX}")

In [ ]:
# Cell 3: Imports and helper functions
import yfinance as yf
import psycopg2
from psycopg2.extras import execute_values
import pandas as pd
import time
from datetime import datetime, timedelta
import traceback

def get_connection():
    return psycopg2.connect(**DB_CONFIG)

def get_missing_assets(conn, asset_class):
    """Get canonical assets that are missing price data."""
    with conn.cursor() as cur:
        if asset_class == 'CRYPTO':
            cur.execute("""
                SELECT canonical_id, ticker 
                FROM fhq_meta.assets
                WHERE asset_class = 'CRYPTO' AND active_flag = true
                AND canonical_id NOT IN (
                    SELECT REPLACE(REPLACE(listing_id, 'LST_', ''), '_XCRYPTO', '') || '-USD'
                    FROM fhq_data.price_series
                    WHERE listing_id LIKE 'LST_%_XCRYPTO'
                )
                ORDER BY ticker
            """)
        elif asset_class == 'EQUITY':
            cur.execute("""
                SELECT canonical_id, ticker 
                FROM fhq_meta.assets
                WHERE asset_class = 'EQUITY' AND active_flag = true
                AND ticker NOT IN (
                    SELECT listing_id FROM fhq_data.price_series
                )
                ORDER BY ticker
            """)
        elif asset_class == 'FX':
            cur.execute("""
                SELECT canonical_id, ticker 
                FROM fhq_meta.assets
                WHERE asset_class = 'FX' AND active_flag = true
                AND ticker NOT IN (
                    SELECT listing_id FROM fhq_data.price_series
                )
                ORDER BY ticker
            """)
        return cur.fetchall()

def fetch_yahoo_data(ticker, start_date):
    """Fetch historical data from Yahoo Finance with retry logic."""
    max_retries = 3
    for attempt in range(max_retries):
        try:
            yf_ticker = yf.Ticker(ticker)
            df = yf_ticker.history(start=start_date, auto_adjust=False)
            if df.empty:
                return None, f"No data returned for {ticker}"
            return df, None
        except Exception as e:
            if 'Too Many Requests' in str(e) or '429' in str(e):
                wait_time = (2 ** attempt) * 30  # 30, 60, 120 seconds
                print(f"  Rate limited, waiting {wait_time}s...")
                time.sleep(wait_time)
            else:
                return None, str(e)
    return None, "Max retries exceeded"

def insert_price_data(conn, listing_id, df):
    """Insert price data into fhq_data.price_series."""
    records = []
    for idx, row in df.iterrows():
        records.append((
            listing_id,
            idx.to_pydatetime(),
            float(row['Open']) if pd.notna(row['Open']) else None,
            float(row['High']) if pd.notna(row['High']) else None,
            float(row['Low']) if pd.notna(row['Low']) else None,
            float(row['Close']) if pd.notna(row['Close']) else None,
            int(row['Volume']) if pd.notna(row['Volume']) else 0,
            float(row.get('Adj Close', row['Close'])) if pd.notna(row.get('Adj Close', row['Close'])) else None
        ))
    
    with conn.cursor() as cur:
        execute_values(cur, """
            INSERT INTO fhq_data.price_series 
            (listing_id, date, open, high, low, close, volume, adj_close)
            VALUES %s
            ON CONFLICT (listing_id, date) DO UPDATE SET
                open = EXCLUDED.open,
                high = EXCLUDED.high,
                low = EXCLUDED.low,
                close = EXCLUDED.close,
                volume = EXCLUDED.volume,
                adj_close = EXCLUDED.adj_close
        """, records)
    conn.commit()
    return len(records)

print("Helper functions loaded.")

In [ ]:
# Cell 4: Check connection and show what needs backfill
try:
    conn = get_connection()
    print("Database connection successful!")
    
    summary = []
    if BACKFILL_CRYPTO:
        crypto_missing = get_missing_assets(conn, 'CRYPTO')
        print(f"\nCRYPTO: {len(crypto_missing)} assets need backfill")
        for c in crypto_missing[:5]:
            print(f"  - {c[1]}")
        if len(crypto_missing) > 5:
            print(f"  ... and {len(crypto_missing) - 5} more")
        summary.append(('CRYPTO', crypto_missing))
    
    if BACKFILL_EQUITY:
        equity_missing = get_missing_assets(conn, 'EQUITY')
        print(f"\nEQUITY: {len(equity_missing)} assets need backfill")
        summary.append(('EQUITY', equity_missing))
    
    if BACKFILL_FX:
        fx_missing = get_missing_assets(conn, 'FX')
        print(f"\nFX: {len(fx_missing)} assets need backfill")
        summary.append(('FX', fx_missing))
    
    conn.close()
except Exception as e:
    print(f"Connection failed: {e}")
    print("\nPlease check your DB_CONFIG values in Cell 2")

In [ ]:
# Cell 5: RUN THE BACKFILL
# This is the main execution cell - run this to start backfilling

def run_backfill():
    conn = get_connection()
    total_success = 0
    total_failed = 0
    failed_assets = []
    
    for asset_class, enabled in [('CRYPTO', BACKFILL_CRYPTO), ('EQUITY', BACKFILL_EQUITY), ('FX', BACKFILL_FX)]:
        if not enabled:
            continue
            
        print(f"\n{'='*60}")
        print(f"BACKFILLING {asset_class}")
        print(f"{'='*60}")
        
        missing = get_missing_assets(conn, asset_class)
        total = len(missing)
        
        for batch_num in range(0, total, BATCH_SIZE):
            batch = missing[batch_num:batch_num + BATCH_SIZE]
            print(f"\nBatch {batch_num // BATCH_SIZE + 1}/{(total + BATCH_SIZE - 1) // BATCH_SIZE}")
            
            for canonical_id, ticker in batch:
                # Determine listing_id format
                if asset_class == 'CRYPTO':
                    base = ticker.split('-')[0]  # BTC-USD -> BTC
                    listing_id = f'LST_{base}_XCRYPTO'
                else:
                    listing_id = ticker
                
                print(f"  [{ticker}] Fetching...", end=' ')
                df, error = fetch_yahoo_data(ticker, START_DATE)
                
                if error:
                    print(f"FAILED: {error}")
                    total_failed += 1
                    failed_assets.append((ticker, error))
                else:
                    try:
                        rows = insert_price_data(conn, listing_id, df)
                        print(f"OK ({rows} rows, {df.index.min().date()} to {df.index.max().date()})")
                        total_success += 1
                    except Exception as e:
                        print(f"INSERT FAILED: {e}")
                        total_failed += 1
                        failed_assets.append((ticker, str(e)))
                
                time.sleep(DELAY_BETWEEN_REQUESTS)
            
            if batch_num + BATCH_SIZE < total:
                print(f"\n  Cooling down for {DELAY_BETWEEN_BATCHES}s...")
                time.sleep(DELAY_BETWEEN_BATCHES)
    
    conn.close()
    
    print(f"\n{'='*60}")
    print(f"BACKFILL COMPLETE")
    print(f"{'='*60}")
    print(f"Success: {total_success}")
    print(f"Failed: {total_failed}")
    
    if failed_assets:
        print(f"\nFailed assets:")
        for ticker, error in failed_assets:
            print(f"  - {ticker}: {error}")

# Uncomment the line below to run
# run_backfill()

In [ ]:
# Cell 6: Verify results
conn = get_connection()
with conn.cursor() as cur:
    cur.execute("""
        SELECT 
            CASE 
                WHEN listing_id LIKE 'LST_%_XCRYPTO' THEN 'CRYPTO'
                WHEN listing_id LIKE '%=X' THEN 'FX'
                ELSE 'EQUITY'
            END as asset_class,
            COUNT(DISTINCT listing_id) as assets_with_data,
            SUM(CASE WHEN MAX(date)::date >= CURRENT_DATE - 1 THEN 1 ELSE 0 END) as fresh_count
        FROM fhq_data.price_series
        GROUP BY 1
        ORDER BY 1
    """)
    print("Current coverage after backfill:")
    print(f"{'Asset Class':<15} {'Assets':<10} {'Fresh':<10}")
    print("-" * 35)
    for row in cur.fetchall():
        print(f"{row[0]:<15} {row[1]:<10} {row[2]:<10}")
conn.close()

## Instructions

1. **Cell 2**: Update `DB_CONFIG` with your Supabase credentials
2. **Cell 2**: Set which asset classes to backfill (CRYPTO, EQUITY, FX)
3. **Cell 4**: Run to verify connection and see what needs backfill
4. **Cell 5**: Uncomment `run_backfill()` and run to start
5. **Cell 6**: Verify results

### Rate Limiting
- Default: 2s between requests, 60s between batches of 10
- If you hit 429 errors, increase `DELAY_BETWEEN_REQUESTS`

### Expected Runtime
- 48 crypto assets: ~30 minutes
- 396 equity assets: ~4 hours
- 25 FX assets: ~15 minutes